In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("NLP").getOrCreate()

In [3]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

In [4]:
sen_df = spark.createDataFrame([
    (0, "Hi I heard about Spark"),
    (1, "I wish java could use classes"),
    (2, "Logistic,regression,models,are,neat")
], ["id", "sentence"])

In [5]:
sen_df.show()

+---+--------------------+
| id|            sentence|
+---+--------------------+
|  0|Hi I heard about ...|
|  1|I wish java could...|
|  2|Logistic,regressi...|
+---+--------------------+



In [6]:
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")
regex_tokenizer = RegexTokenizer(inputCol="sentence", outputCol="words", pattern="\\W")

In [7]:
count_tokens = udf(lambda words:len(words), IntegerType())

In [8]:
tokenized = tokenizer.transform(sen_df)
tokenized.show()

+---+--------------------+--------------------+
| id|            sentence|               words|
+---+--------------------+--------------------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|
|  1|I wish java could...|[i, wish, java, c...|
|  2|Logistic,regressi...|[logistic,regress...|
+---+--------------------+--------------------+



In [9]:
tokenized.withColumn("tokens", count_tokens(col("words"))).show()

+---+--------------------+--------------------+------+
| id|            sentence|               words|tokens|
+---+--------------------+--------------------+------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|     5|
|  1|I wish java could...|[i, wish, java, c...|     6|
|  2|Logistic,regressi...|[logistic,regress...|     1|
+---+--------------------+--------------------+------+



In [10]:
rg_tokenized = regex_tokenizer.transform(sen_df)

In [11]:
rg_tokenized.withColumn("tokens", count_tokens(col("words"))).show()

+---+--------------------+--------------------+------+
| id|            sentence|               words|tokens|
+---+--------------------+--------------------+------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|     5|
|  1|I wish java could...|[i, wish, java, c...|     6|
|  2|Logistic,regressi...|[logistic, regres...|     5|
+---+--------------------+--------------------+------+



In [12]:
from pyspark.ml.feature import StopWordsRemover

In [13]:
sentenceDataFrame = spark.createDataFrame([
    (0, ["I", "saw", "the", "green", "horse"]),
     (0, ["Mary", "had", "a", "little", "lamb"])
], ["id", "tokens"])

In [14]:
remover = StopWordsRemover(inputCol="tokens", outputCol="filtered")

In [15]:
remover.transform(sentenceDataFrame).show()

+---+--------------------+--------------------+
| id|              tokens|            filtered|
+---+--------------------+--------------------+
|  0|[I, saw, the, gre...| [saw, green, horse]|
|  0|[Mary, had, a, li...|[Mary, little, lamb]|
+---+--------------------+--------------------+



In [16]:
#N-GRAM
from pyspark.ml.feature import NGram

In [20]:
wordDataFrame = spark.createDataFrame([
    (0, ["Hi", "I", "heard", "about", "Spark"]),
    (1, ["I", "wish", "Java", "use", "classes"]),
    (2, ["Logistic", "regression", "models", "are", "nice"])
], ["id", "words"])

In [21]:
ngram = NGram(n=2, inputCol="words", outputCol="grams")

In [23]:
ngram.transform(wordDataFrame).show(truncate=False)

+---+-----------------------------------------+--------------------------------------------------------------+
|id |words                                    |grams                                                         |
+---+-----------------------------------------+--------------------------------------------------------------+
|0  |[Hi, I, heard, about, Spark]             |[Hi I, I heard, heard about, about Spark]                     |
|1  |[I, wish, Java, use, classes]            |[I wish, wish Java, Java use, use classes]                    |
|2  |[Logistic, regression, models, are, nice]|[Logistic regression, regression models, models are, are nice]|
+---+-----------------------------------------+--------------------------------------------------------------+

